In [1]:
from elasticsearch import Elasticsearch,client 
es = Elasticsearch()

In [51]:
def get_es_batch():
    page = es.search(
            index = 'pubmed_sents',
            doc_type = 'articles',
            scroll = '3m',
            size = 32,
            body = {
                'query':{'match_all':{}}
            }
        )
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']

    # Start scrolling
    while (scroll_size > 0):
        page = es.scroll(scroll_id = sid, scroll = '2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        image = [i['_source']['lineNum'] for i in page['hits']['hits']]
#         image = [i['_source']['label'] for i in page['hits']['hits']]
        for i in image:
            yield i

In [ ]:
dataset = tf.data.Dataset().batch(32).from_generator(get_es_batch,
                                           output_types= tf.float32, 
                                           output_shapes=(tf.TensorShape([32, None])))

In [14]:
iter = dataset.make_initializable_iterator()
el = iter.get_next()
with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))
    print(sess.run(el))
    print(sess.run(el))

{u'_id': u'432435',
 u'_index': u'pubmed_sents',
 u'_score': 1.0,
 u'_source': {u'docID': u'12626416',
  u'lineNum': 432435,
  u'sentence': u'the glycan moiety of human recombinant gonadotrophins (r-hfsh, r-hlh, and r-hcg) produced in cho cell lines has been characterized by a combination of chromatographic and mass spectrometric techniques, including both matrix-assisted laser desorption ionization and electrospray.'},
 u'_type': u'articles'}